In [78]:
import pandas as pd
import numpy as np
from itertools import combinations
import re
import json
from math import ceil
import tiktoken
import time
import os
import openai
from statistics import mean
from json import JSONDecodeError
from copy import deepcopy

In [2]:
DIR = os.getenv('KEA_BASE_DIR')

## Profession Segmentation

In [3]:
const_myneta13 = pd.read_csv(f'{DIR}/2013 Elections/winners2013MyNeta.csv',index_col=0)
const_myneta18 = pd.read_csv(f'{DIR}/2018 Elections/winners2018MyNeta.csv',index_col=0)
const_myneta23 = pd.read_csv(f'{DIR}/2023 Elections/winners2023MyNeta.csv',index_col=0)
candid_myneta13 = pd.read_csv(f'{DIR}/2013 Elections/candidates2013MyNeta.csv',index_col=0)
candid_myneta18 = pd.read_csv(f'{DIR}/2018 Elections/candidates2018MyNeta.csv',index_col=0)
candid_myneta23 = pd.read_csv(f'{DIR}/2023 Elections/candidates2023MyNeta.csv',index_col=0)

In [4]:
self_prof_23 = list(candid_myneta23.self_profession.unique()) + list(const_myneta13.self_profession.unique())
self_prof_13 = list(candid_myneta13.self_profession.unique()) + list(const_myneta23.self_profession.unique())
self_prof_18 = list(candid_myneta18.self_profession.unique()) + list(const_myneta18.self_profession.unique())

spouse_prof_23 = list(candid_myneta23.spouse_profession.unique()) + list(const_myneta13.spouse_profession.unique())
spouse_prof_13 = list(candid_myneta13.spouse_profession.unique()) + list(const_myneta23.spouse_profession.unique())
spouse_prof_18 = list(candid_myneta18.spouse_profession.unique()) + list(const_myneta18.spouse_profession.unique())

temp_list = self_prof_23 + self_prof_13 + self_prof_18 + spouse_prof_23 + spouse_prof_13 + spouse_prof_18
total_prof = list(set(temp_list))
len(total_prof)

4233

In [15]:
categories_raw = """Politics / Public Service
Business / Entrepreneurship
Law / Judiciary
Education / Academia
Medicine / Healthcare
Engineering / Technology
Agriculture / Farming
Media / Journalism
Arts / Entertainment
Social Work / Activism
Sports / Athletics
Military / Defense
Information Technology
Banking / Finance
Environmental Conservation
Transportation / Logistics
Science / Research
Hospitality / Tourism
Construction / Real Estate
Religion / Spirituality
Law Enforcement / Security
Consulting / Advisory"""

In [16]:
categories = [str(i+1) + '. ' + categories_raw.split('\n')[i] for i in range(len(categories_raw.split('\n')))]
categories = '\n'.join(categories)
categories

'1. Politics / Public Service\n2. Business / Entrepreneurship\n3. Law / Judiciary\n4. Education / Academia\n5. Medicine / Healthcare\n6. Engineering / Technology\n7. Agriculture / Farming\n8. Media / Journalism\n9. Arts / Entertainment\n10. Social Work / Activism\n11. Sports / Athletics\n12. Military / Defense\n13. Information Technology\n14. Banking / Finance\n15. Environmental Conservation\n16. Transportation / Logistics\n17. Science / Research\n18. Hospitality / Tourism\n19. Construction / Real Estate\n20. Religion / Spirituality\n21. Law Enforcement / Security\n22. Consulting / Advisory'

In [17]:
categ_dict = {(i+1):categories_raw.split('\n')[i] for i in range(len(categories_raw.split('\n')))}
categ_dict[101] = 'Unknown'
categ_dict[201] = "Cound't Classify"
categ_dict[301] = 'Not Given'
categ_dict

{1: 'Politics / Public Service',
 2: 'Business / Entrepreneurship',
 3: 'Law / Judiciary',
 4: 'Education / Academia',
 5: 'Medicine / Healthcare',
 6: 'Engineering / Technology',
 7: 'Agriculture / Farming',
 8: 'Media / Journalism',
 9: 'Arts / Entertainment',
 10: 'Social Work / Activism',
 11: 'Sports / Athletics',
 12: 'Military / Defense',
 13: 'Information Technology',
 14: 'Banking / Finance',
 15: 'Environmental Conservation',
 16: 'Transportation / Logistics',
 17: 'Science / Research',
 18: 'Hospitality / Tourism',
 19: 'Construction / Real Estate',
 20: 'Religion / Spirituality',
 21: 'Law Enforcement / Security',
 22: 'Consulting / Advisory',
 101: 'Unknown',
 201: "Cound't Classify",
 301: 'Not Given'}

In [82]:
command1 = f"""I am sending a list of responses submitted by people which contains information about their profession. Classify them
into the following categories:  

{categories}

I want you to send only the category numbers on new lines for all professions. Do not send the professions
Do not use any other category than those given to you.
Do not change or mix any of the categories. If some profession lies into multiple categories, list those categories' numbers on the same line separated by a comma.
Don't send anything else in the response. If you cannot classify a profession into any category listed, use number 101 as the category number.
If you cannot understand what's the profession, use number 201 as the category number.
If the profession is 'Not Given', then use number 301 as the category number.
The professions given to you are of Indian people. Some of the professions might be local language words which are phonetically spelled in english.
So, take that in account too, when classifying the professions.
Also, don't change the sequence of category numbers and professions.

Example:

example_profession1
category1, category2
example_profession2
category2
example profession3
category1


List of responses:

"""

command2 = f"""I am sending a list of responses submitted by people which contains information about their profession. Classify them
into the following categories:  

{categories}

I want you to send the output in json format only. The keys will be the professions I send you, and their corresponding values will be a list of category numbers that you have classified the profession into.
The profession can be classified into multiple categories.
Do not use any other category than those given to you.
Do not change or mix any of the categories.
Don't send anything else in the response. If you cannot classify a profession into any category listed, use number 101 as the category number.
If you cannot understand what's the profession, use number 201 as the category number.
If the profession is 'Not Given', then use number 301 as the category number.
The professions given to you are of Indian people. Some of the professions might be local language words which are phonetically spelled in english.
So, take that in account too, when classifying the professions.
Also, don't change the sequence of category numbers and professions.


List of responses:

"""

command3 = f"""I am sending a list of responses submitted by people which contains information about their profession. Classify them
into the following categories:  

{categories}

I want you to send the output in json format only. The keys will be the professions I send you, and their corresponding values will be a list of category numbers that you have classified the profession into.
The professions will be sent to you in double quotes. Use them as as they are for the keys. Do not change their wording, spacing or anything.
The profession can be classified into multiple categories.
Do not use any other category than those given to you.
Do not change or mix any of the categories.
Don't send anything else in the response. If you cannot classify a profession into any category listed, use number 101 as the category number.
If you cannot understand what's the profession, use number 201 as the category number.
If the profession is 'Not Given', then use number 301 as the category number.
The professions given to you are of Indian people. Some of the professions might be local language words which are phonetically spelled in english.
So, take that in account too, when classifying the professions.
Also, don't change the sequence of category numbers and professions.
Do not send anything else other than the JSON format."""

command4 = f"""I am sending a single response submitted by a person which contains information about their profession. Classify the profession
into the following categories:  

{categories}

I want you to send the output in json format only. The key will be the profession I send you, and its corresponding value will be a list of category numbers that you have classified the profession into.
The profession will be sent to you in double quotes. Use it as as it is as the key. Do not change it's wording, spacing or anything.
The profession can be classified into multiple categories.
Do not use any other category than those given to you.
Do not change or mix any of the categories.
Don't send anything else in the response. If you cannot classify a profession into any category listed, use number 101 as the category number.
If you cannot understand what's the profession, use number 201 as the category number.
If the profession has value "Not Given", then use number 301 as the category number.
The profession given to you is of Indian people. The profession might be local language words which are phonetically spelled in english.
So, take that in account too, when classifying the profession.
Do not send anything else other than the JSON format."""


In [83]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
len(encoding.encode(command3))

399

In [89]:
import os
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

new_dict = {}
completion = ''
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
length = []
BATCH_SIZE = 50

for i in range(ceil(len(total_prof)/BATCH_SIZE)):
    before = time.perf_counter()
    prev_response = completion
    print(i-1,'number of runs completed') if (i%100==0 and i!=0) else None
    success, break_outer = False, False
    count = 0
    
    while (not success) and (count<2):
        try:
            start = i*BATCH_SIZE
            end = len(total_prof) if (start + BATCH_SIZE > len(total_prof)) else start + BATCH_SIZE
            
            string = '"' + '"\n"'.join(total_prof[start:end]) + '"'
            print(string,'\n\n')
            completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                {"role": "system", "content": f"You are a helpful assistant and also a python programmer. {command3}"},
                {"role": "user", "content": "Horticulture and Business"},
                {"role": "assistant", "content": '{"Horticulture and Business": [7, 2]}'},
                {"role": "user", "content": "BTM Investments Pvt Ltd Hubballi Director BTM forging Pvt Ltd Director Veetech Valves Pvt Ltd Director S R Process Controls Pvt Ltd Partner Future Auto Solutions Hubballi"},
                {"role": "assistant", "content": '{"BTM Investments Pvt Ltd Hubballi Director BTM forging Pvt Ltd Director Veetech Valves Pvt Ltd Director S R Process Controls Pvt Ltd Partner Future Auto Solutions Hubballi": [2]}'},
                {"role": "user", "content": string}
              ]
            )
        except:
            count += 1
            print('count',count,'for run number',i)
            time.sleep(60) if count == 1 else None
            break_outer = True if count > 1 else False
        else:
            try:
                temp_dict = dict(json.loads(completion['choices'][0]['message']['content']))
                assert len(list(temp_dict.keys()))==(end-start)
                new_dict.update({f"{total_prof[j]}":list(temp_dict.values())[j-start] for j in range(start,end)}) # = str_dict + completion['choices'][0]['message']['content'].strip() + '\n'
            except AssertionError:
                count += 1
                print(f"Assertion failed for {i}th run for value {total_prof[start:end]} and response as {completion['choices'][0]['message']['content']}\n")
            except JSONDecodeError:
                count += 1
                print(f"JSON decoding failed for {i}th run for value {total_prof[i]} and response as {completion['choices'][0]['message']['content']}\n")
            else:
                success = True
                prev_response = completion
                time.sleep(2)
    length.append(time.perf_counter()-before)
    if break_outer:
        print('breaking at run',i)
        break

print("average time taken",mean(length))

"Press and Editor of Echarike 24 7"
"Urdu Teacher"
"Housewife and Agriculturlist"
"Photographer"
"Agriculture,"
"BDA Retired Employee"
"House wife and agriculture"
"Self Employment and Buisness"
"Agricultural Business"
"business trading"
"BEAUTION"
"Advocate Profession"
"SMALL SCALE LABOUR"
"House Wife and Municipality Member in Mundaragi"
"agriculter and editor for paper"
"Social service and coolie"
"Tenacity Tailor housewife"
"Salesman"
"Retired officer KEB"
"Cine Artist Consultant"
"House Wife and agriculture"
"Self Employement"
"Business-Real Estate"
"Bamboo Business"
"Holly Lawmaker"
"H H Work"
"CONTRACTER"
"Homemaker and Business"
"Farmer and Editor in sogadu newspaper,chickpete"
"Byatarayanapura Assembly Constituency,Karnataka"
"Hobbyist -Electrician"
"Buisiness"
"Medical Doctor"
"Permanent job"
"Household, Social Service"
"Farming ,Constructed items"
"ADVOCATE"
"Social work"
"Business, Agriculturist and other Business(Sales and Services of Agricultural Equipment)."
"LAWYER , KA

In [101]:
data_json = json.dumps(temp_dict)
with open('profession.json','w',encoding='utf-8') as file:
    file.write(data_json)

## Education Segmentation

In [22]:
edu_categ_raw = """Illiterate: Candidates who have not received any formal education and are illiterate.
Literate: Candidates who are educated and can read and write but have not completed formal schooling beyond basic literacy.
Primary Education: Candidates who have completed primary education up to the 5th standard.
Secondary Education: Candidates who have completed secondary education up to the 10th standard.
Higher Secondary Education: Candidates who have completed higher secondary education up to the 12th standard.
Diploma Holders: Candidates who have completed diploma courses in various fields like engineering, pharmacy, nursing, etc.
Graduates: Candidates who hold a Bachelor's degree in any discipline.
Postgraduates: Candidates who have completed a Master's degree in any discipline.
Doctorates: Candidates who have obtained a Doctorate (Ph.D.) in any field.
Technical/Professional Courses: Candidates who have completed specific technical or professional courses, such as Chartered Accountancy (CA), Company Secretary (CS), and various vocational courses.
Vocational Training: Candidates who have undergone vocational training programs, such as Industrial Training Institutes (ITI) or other skill development courses.
Non-Traditional Education: Candidates who have pursued non-traditional forms of education, such as distance learning, open universities, or online courses."""

edu_categ = [str(i+1) + '. ' + edu_categ_raw.split('\n')[i] for i in range(len(edu_categ_raw.split('\n')))]
edu_categ = "\n".join(edu_categ)
edu_categ

"1. Illiterate: Candidates who have not received any formal education and are illiterate.\n2. Literate: Candidates who are educated and can read and write but have not completed formal schooling beyond basic literacy.\n3. Primary Education: Candidates who have completed primary education up to the 5th standard.\n4. Secondary Education: Candidates who have completed secondary education up to the 10th standard.\n5. Higher Secondary Education: Candidates who have completed higher secondary education up to the 12th standard.\n6. Diploma Holders: Candidates who have completed diploma courses in various fields like engineering, pharmacy, nursing, etc.\n7. Graduates: Candidates who hold a Bachelor's degree in any discipline.\n8. Postgraduates: Candidates who have completed a Master's degree in any discipline.\n9. Doctorates: Candidates who have obtained a Doctorate (Ph.D.) in any field.\n10. Technical/Professional Courses: Candidates who have completed specific technical or professional cours

In [23]:
edu_categ_dict = {(i+1):edu_categ_raw.split('\n')[i] for i in range(len(edu_categ_raw.split('\n')))}
edu_categ_dict[101] = 'Unknown'
edu_categ_dict

{1: 'Illiterate: Candidates who have not received any formal education and are illiterate.',
 2: 'Literate: Candidates who are educated and can read and write but have not completed formal schooling beyond basic literacy.',
 3: 'Primary Education: Candidates who have completed primary education up to the 5th standard.',
 4: 'Secondary Education: Candidates who have completed secondary education up to the 10th standard.',
 5: 'Higher Secondary Education: Candidates who have completed higher secondary education up to the 12th standard.',
 6: 'Diploma Holders: Candidates who have completed diploma courses in various fields like engineering, pharmacy, nursing, etc.',
 7: "Graduates: Candidates who hold a Bachelor's degree in any discipline.",
 8: "Postgraduates: Candidates who have completed a Master's degree in any discipline.",
 9: 'Doctorates: Candidates who have obtained a Doctorate (Ph.D.) in any field.',
 10: 'Technical/Professional Courses: Candidates who have completed specific tec

In [7]:
abbreviations = """
Here's a comprehensive list of degrees and their abbreviations commonly used in India:

Bachelor's Degrees:
    B.A. - Bachelor of Arts
    B.Sc. - Bachelor of Science
    B.Com. - Bachelor of Commerce
    B.Tech. - Bachelor of Technology
    B.E. - Bachelor of Engineering
    BBA - Bachelor of Business Administration
    BCA - Bachelor of Computer Applications
    B.Arch. - Bachelor of Architecture
    B.Pharma - Bachelor of Pharmacy
    B.Ed. - Bachelor of Education
    BFA - Bachelor of Fine Arts
    B.S.W. - Bachelor of Social Work
    B.Lib. - Bachelor of Library Science
    B.P.Ed. - Bachelor of Physical Education
    B.H.M. - Bachelor of Hotel Management
    B.Sc. (Nursing) - Bachelor of Science in Nursing
    B.A.M.S. - Bachelor of Ayurvedic Medicine and Surgery
    B.H.M.S. - Bachelor of Homeopathic Medicine and Surgery
    B.U.M.S. - Bachelor of Unani Medicine and Surgery
    B.D.S. - Bachelor of Dental Surgery
    B.V.Sc. - Bachelor of Veterinary Science
    
Master's Degrees:
    M.A. - Master of Arts
    M.Sc. - Master of Science
    M.Com. - Master of Commerce
    M.Tech. - Master of Technology
    M.E. - Master of Engineering
    MBA - Master of Business Administration
    MCA - Master of Computer Applications
    M.Arch. - Master of Architecture
    M.Pharma - Master of Pharmacy
    M.Ed. - Master of Education
    MFA - Master of Fine Arts
    M.S.W. - Master of Social Work
    M.Lib. - Master of Library Science
    M.P.Ed. - Master of Physical Education
    M.Sc. (Nursing) - Master of Science in Nursing
    M.D. - Doctor of Medicine (Medicine)
    M.S. - Master of Surgery
    M.R.C.P. - Member of the Royal College of Physicians
    M.R.C.S. - Member of the Royal College of Surgeons
    
Doctoral Degrees:
    Ph.D. - Doctor of Philosophy
    D.Sc. - Doctor of Science
    D.Litt. - Doctor of Literature
    D.Lit. - Doctor of Literature
    D.M. - Doctor of Medicine (Specialized Medical Degree)
    
Professional Degrees:
    M.B.B.S. - Bachelor of Medicine and Bachelor of Surgery
    LLB - Bachelor of Laws
    LLM - Master of Laws
    CA - Chartered Accountant
    CS - Company Secretary
    ICWA - Institute of Cost and Works Accountants of India
    
Other Degrees:
    D.Pharm. - Diploma in Pharmacy
    D.Ed. - Diploma in Education
    DSW - Diploma in Social Work
    D.Lib. - Diploma in Library Science
    DPT - Diploma in Physical Therapy
    DHMS - Diploma in Homeopathic Medicine and Surgery
    DOTT - Diploma in Operation Theater Technology
    DMLT - Diploma in Medical Laboratory Technology
    DCH - Diploma in Child Health
    DMRD - Diploma in Medical Radiodiagnosis
"""

In [8]:
edu_command = f"""You will be provided with a list of electoral candidates from Indian Assembly elections along with their education details. Classify each candidate's education level into one of the predefined categories based on their highest qualification achieved. Use the following categories for classification:

{edu_categ_dict}

Note - The education values will be sent to you in double quotes with each candidates education on a new line with double quotes.
Only use the category numbers in the output. Don't send the categories themselves in the output.
If the candidate's education is not explicitly mentioned, and you are unable to determine their education level, please classify them as "Unknown." with category number 101.
Please provide the classification for each candidate in JSON format, where keys are the cabdidate names and values are their corresponding education category numbers.
Classify each candidate's education level into one of the predefined categories based on their highest qualification achieved.
If a candidate has multiple qualifications, consider their highest level of education attained."""

In [21]:
edu = []

edu_23 = list(pd.Series(candid_myneta23.education.unique()).fillna('Unknown').values) + list(pd.Series(const_myneta13.education.unique()).fillna('Unknown').values)
edu_13 = list(pd.Series(candid_myneta13.education.unique()).fillna('Unknown').values) + list(pd.Series(const_myneta23.education.unique()).fillna('Unknown').values)
edu_18 = list(pd.Series(candid_myneta18.education.unique()).fillna('Unknown').values) + list(pd.Series(candid_myneta18.education.unique()).fillna('Unknown').values)

temp_list = edu_23 + edu_13 + edu_18
edu = list(set(temp_list))
len(edu)

7271

In [10]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
len(encoding.encode(edu_command + '"' + '"\n"'.join(edu[:50]) + '"'))

1427

In [63]:

new_dict = {}
completion = ''
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
length = []
asserts = []
decodes = []
BATCH_SIZE = 50
###################################### CHANGE BEFORE PRODUCTION ########################################
for i in range(ceil(len(edu)/BATCH_SIZE)): #116,125,133
###################################### CHANGE BEFORE PRODUCTION ########################################
    before = time.perf_counter()
    prev_response = completion
    print(i-1,'number of runs completed') if (i%100==0 and i!=0) else None
    success, break_outer = False, False
    count = 0
    
    
    while (not success) and (count<2):
        try:
            start = i*BATCH_SIZE
            end = len(edu) if (start + BATCH_SIZE > len(edu)) else start + BATCH_SIZE
            print(f"{i}th run for count {count}, start {start} and end {end}")
            
            string = '"' + '"\n"'.join(temp_list) + '"'
            completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
                
              messages=[
                {"role": "system", "content": f"You are a helpful assistant and also a python programmer. {edu_command}"},
                {"role": "user", "content": "vijaya collage"},
                {"role": "assistant", "content": '{"vijaya collage": 7}'},
                {"role": "user", "content": "B.COM DEGREE FROM SB HIGH SCHOOL KALABURAGI AND SB COLLEGE KALABURAGI IN 1970-71,L.L.B FROM SSL LAW COLLEGE KALBURAGI IN 1973-74"},
                {"role": "assistant", "content": '{"B.COM DEGREE FROM SB HIGH SCHOOL KALABURAGI AND SB COLLEGE KALABURAGI IN 1970-71,L.L.B FROM SSL LAW COLLEGE KALBURAGI IN 1973-74": 7}'},
                {"role": "user", "content": "5th Pass From Marathi Primary School Kuduchi 1992"},
                {"role": "assistant", "content": '{"5th Pass From Marathi Primary School Kuduchi 1992": 4}'},
                {"role": "user", "content": "B.Ed from Kamala Nehru College Bengalore in 2007, B.A from B K College Belagavi in 2004"},
                {"role": "assistant", "content": '{"B.Ed from Kamala Nehru College Bengalore in 2007, B.A from B K College Belagavi in 2004": 7}'}, 
                {"role": "user", "content": string}
              ]
            )
            # print(completion)
        except:
            count += 1
            print('count',count,'for run number',i)
            time.sleep(60) if count == 1 else None
            break_outer = True if count > 1 else False
        else:
            try:
                temp_dict = dict(json.loads(completion['choices'][0]['message']['content']))
                assert len(list(temp_dict.keys()))==(end-start)
                new_dict.update({edu[j]:list(temp_dict.values())[j-start] for j in range(start,end)}) # = str_dict + completion['choices'][0]['message']['content'].strip() + '\n'
            except AssertionError:
                count += 1
                asserts.append(i)
                print(f"Assertion failed for {i}th run for count {count}, start {start} and end {end}")
            except JSONDecodeError:
                count += 1
                decodes.append(i)
                print(f"JSON decoding failed for {i}th run for count {count}, start {start} and end {end}")
            else:
                success = True
                prev_response = completion
                time.sleep(1)
    length.append(time.perf_counter()-before)
    if break_outer:
        print('breaking at run',i)
        break

print("average time taken for 20 runs",mean(length))

133th run for count 0, start 6650 and end 6700
{'10 th fail govt school gollahalli,tumkur district madhugiri': 4, '10th Fail, Govt High School, Chintamani': 4, '2nd PUC from B.A.J.S.S.P.U.College Rannebanur 2007': 5, '2nd PUC, Govt. Pre-University College, Arasikere Hassan Dist.': 5, '5th std,medical uppunda,kundapura taluk,udupi district': 3, '7 th standard Government urdu school kolar 1966-67': 3, '7TH STANDARD GOVERNMENT HIGHER PRIMARY SCHOOL DONABAGATT BHADRAVATI TALUK': 3, '7th Standard in Govt. High School, Arkere Shrirangapattana Taluk (1994-95).': 4, '7th pass urdu government high school,alipura in 1996-97': 4, '7th standard, MVVS, Afzalpur, year of passing - 1988': 4, '7th std,ahamadhiya primary school,mandi mohalla, mysore': 4, '9th Pass Dayananda Boys High School Banglore-78 Karnataka In Year 1995-96': 4, '9th from Govt. Urdu School Atnoor': 4, '9th std.,Silver jublee high school, New town, bhadravathi': 4, 'B.Com. From Canara College Mangalore University 2000': 7, 'B.Sc. fr

In [67]:
data_json = json.dumps(temp_dict)
with open('education.json','w',encoding='utf-8') as file:
    file.write(data_json)

### Cleaning the JSONs

In [7]:
with open('education.json','r',encoding='utf-8') as file:
    education = dict(json.loads(file.read()))
with open('profession.json','r',encoding='utf-8') as file:
    profession = dict(json.loads(file.read()))

In [34]:
for key in profession.keys():
    temp = profession[key]
    temp = [categ_dict[value] for value in temp]
    profession[key] = temp

In [35]:
for key in education.keys():
    temp = education[key]
    temp = edu_categ_dict[temp].split(':')[0]
    education[key] = temp

In [9]:
with open('education.json','w',encoding='utf-8') as file:
    file.write(json.dumps(education))
with open('profession.json','w',encoding='utf-8') as file:
    file.write(json.dumps(profession))

## Resolving Name conflicts for candidates' names

The names from the MyNeta dataset and other votes dataset were slightly different in spellings, positions, abbreviations, titles, etc.
So, GPT 3.5 Turbo was used to match similar names for candidates in a particular constituency in a particular year.

In [70]:
const_myneta13 = pd.read_csv(f'{DIR}/2013 Elections/winners2013MyNeta.csv',index_col=0)
const_myneta18 = pd.read_csv(f'{DIR}/2018 Elections/winners2018MyNeta.csv',index_col=0)
const_myneta23 = pd.read_csv(f'{DIR}/2023 Elections/winners2023MyNeta.csv',index_col=0)
candid_myneta13 = pd.read_csv(f'{DIR}/2013 Elections/candidates2013MyNeta.csv',index_col=0)
candid_myneta18 = pd.read_csv(f'{DIR}/2018 Elections/candidates2018MyNeta.csv',index_col=0)
candid_myneta23 = pd.read_csv(f'{DIR}/2023 Elections/candidates2023MyNeta.csv',index_col=0)
candid_opencity13 = pd.read_csv(f'{DIR}/2013 Elections/candidates2013OpenCity.csv',index_col=0)
const_opencity13 = pd.read_csv(f'{DIR}/2013 Elections/constituency2013OpenCity.csv',index_col=0)
candid_eci18 = pd.read_csv(f'{DIR}/2018 Elections/candidates2018ECI.csv',index_col=0)
const_eci18 = pd.read_csv(f'{DIR}/2018 Elections/constituency2018ECI.csv',index_col=0)
candid_ndtv23 = pd.read_csv(f'{DIR}/2023 Elections/candidates2023NDTV.csv',index_col=0)
const_ndtv23 = pd.read_csv(f'{DIR}/2023 Elections/constituency2023NDTV.csv',index_col=0)

In [71]:
candid = {
    2013:{},
    2018:{},
    2023:{}
}

for constituency in const_myneta13.constituency.unique():
    temp_dict = {}
    myneta = [name for name in candid_myneta13.loc[candid_myneta13.constituency==constituency,'name']]
    opencity = [name for name in candid_opencity13.loc[candid_opencity13.constituency==constituency,'name']]
    temp_dict['myneta'] = myneta
    temp_dict['opencity'] = opencity
    candid[2013][constituency] = temp_dict
    temp_dict = {}
    myneta = [name for name in candid_myneta18.loc[candid_myneta18.constituency==constituency,'name']]
    eci = [name for name in candid_eci18.loc[candid_eci18.constituency==constituency,'name']]
    temp_dict['myneta'] = myneta
    temp_dict['eci'] = eci
    candid[2018][constituency] = temp_dict
    temp_dict = {}
    myneta = [name for name in candid_myneta23.loc[candid_myneta23.constituency==constituency,'name']]
    ndtv = [name for name in candid_ndtv23.loc[candid_ndtv23.constituency==constituency,'name']]
    temp_dict['myneta'] = myneta
    temp_dict['ndtv'] = ndtv
    candid[2023][constituency] = temp_dict
    

In [72]:
with open('candid_names.json','w',encoding='utf-8') as file:
    json.dump(candid, file)

In [73]:
with open('candid_names.json','r',encoding='utf-8') as file:
    candid = json.load(file)

In [74]:
name_command= f"""I am sending you two lists of names of people. You must match the names of people in the first list to the corresponding similar names in the second list.
Give the output in a json format where the keys are the names in the first list and their corresponding values are their corresponding similar names from the second list.
Note - You only need to exhaust the first list of names. That is, each name in the first list must match at most one similar name from the second list. However, vice versa need not be true in case that the second list has more names that the first.
For a particular name in the first list, if you can't find a similar name in the second list, you must use its value in the json as "Unknown".
Don't send anything else in the output other than the json because I will use python to convert it into a dictionary.
"""
input_ex1 = f"""list1 = {candid['2018']['Nippani']['myneta']}

list1 = {candid['2018']['Nippani']['eci']}"""

input_ex2 = f"""list1 = {candid['2018']['Anekal']['myneta']}

list1 = {candid['2018']['Anekal']['eci']}"""

input_ex3 = f"""list1 = {candid['2023']['Dharwad']['myneta']}

list1 = {candid['2023']['Dharwad']['ndtv']}"""

In [79]:
new_dict = {year:{} for year in list(candid.keys())} 
completion = ''
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
openai.api_key = os.getenv('OPENAI_API_KEY')
length = []
asserts = []
decodes = []
error_dict = deepcopy(new_dict) ############################### change before production
###################################### CHANGE BEFORE PRODUCTION ########################################
for year in list(candid.keys()): ############################### change before production -- add list(candid.keys())
###################################### CHANGE BEFORE PRODUCTION ########################################
    constituencies = list(candid[year].keys())
    list2_key = 'opencity' if year=='2013' else 'eci' if year=='2018' else 'ndtv'
    const_dict = {constituency:{} for constituency in constituencies}
    const_error_dict = deepcopy(const_dict)
    for i in range(len(constituencies)):
        before = time.perf_counter()
        prev_response = completion
        print(i+1,'th constituency starting') if (i%75==0 and i!=0) else None
        success, break_outer = False, False
        count = 0

        while (not success) and (count<2): 
            try:
                input_string = f"""list1 = {candid[year][constituencies[i]]['myneta']}\n\nlist2 = {candid[year][constituencies[i]][list2_key]}"""

                completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                    
                messages=[
                    {"role": "system", "content": f"You are a helpful assistant and also a python programmer. {name_command}"},
                    {"role": "user", "content": input_ex1},
                    {"role": "assistant", "content": '{\n    "Anil Shivaji Kamate": "Anil Shivaji Kamate",\n    "Jayaram Parasharam Mirajakarkaka": "Jayaram Parasharam Mirajakarkaka",\n    "Kakasaheb Pandurang Patil": "Kakaso Pandurang Patil",\n    "Kamat Ramesh Ishwar": "Kamat Ramesh Ishwar",\n    "Rohini Shrimant Dixit": "Rohini Shrimant Dixit",\n    "Sachin Kallappa Madhale": "Sachin Kallappa Madhale",\n    "Sambhaji Bapuso Thoravat": "Sambhaji Bapuso Thoravat",\n    "Sharad Ramagonda Patil": "Sharad Ramagowda Patil",\n    "Shashikala Annasaheb Jolle": "Jolle Shashikala Annasaheb",\n    "Vijaylakshmi Appasaheb Kurane": "Vijaylakshmi Appasaheb Kurane"\n}'},
                    {"role": "user", "content": input_ex2},
                    {"role": "assistant", "content": '{\n    "A.Narayanaswamy": "A.Narayanaswamy",\n    "Amaresh.Y": "Amaresh.Y",\n    "B.Shivanna": "B.Shivanna",\n    "C.Narayanaswamy": "C.Narayanaswamy",\n    "D.Mahadesh": "D.Mahadesh",\n    "G. Srinivas": "G. Srinivas",\n    "M.Muniyallamma": "M.Muniyallamma",\n    "Munikrishnappa": "Munikrishnappa",\n    "Venkateshappa": "Venkateshappa",\n    "Y.Ravindra": "Y.Ravindra"\n}'},
                    {"role": "user", "content": input_string}
                ]
                )
            except:
                count += 1
                print('count',count,'for run number',i)
                time.sleep(90) if count == 1 else None
                # break_outer = True if count > 1 else False
            else:
                try:
                    temp_dict = dict(json.loads(completion['choices'][0]['message']['content']))
                    assert len(np.setxor1d(list(temp_dict.keys()),candid[year][constituencies[i]]['myneta']))==0
                    assert len(list(temp_dict))==len(set(candid[year][constituencies[i]]['myneta']))
                    const_dict[constituencies[i]].update(temp_dict) # 
                except AssertionError:
                    const_error_dict[constituencies[i]].update({'setxor':np.setxor1d(list(temp_dict.keys()),candid[year][constituencies[i]]['myneta']),
                                             'keys':list(temp_dict.keys()),
                                             'myneta':set(candid[year][constituencies[i]]['myneta'])})
                    count += 1
                    asserts.append(i)
                    print(f"Assertion failed for {i+1}th run for count {count}, for constituency {constituencies[i]}")
                except JSONDecodeError:
                    const_error_dict[constituencies[i]].update({'completion':completion['choices'][0]['message']['content']})
                    count += 1
                    decodes.append(i)
                    print(f"JSON decoding failed for {i+1}th run for count {count}, for constituency {constituencies[i]}")
                else:
                    success = True
                    prev_response = completion
                    time.sleep(3)
        length.append(time.perf_counter()-before)
        if break_outer:
            print('breaking at run',i)
            break
    new_dict[year].update(const_dict)


print("average time taken for 1 constituency",mean(length))

76 th constituency starting
151 th constituency starting
76 th constituency starting
count 1 for run number 113
JSON decoding failed for 137th run for count 1, for constituency Chikkaballapur
151 th constituency starting
Assertion failed for 154th run for count 1, for constituency Gulbarga Uttar
Assertion failed for 164th run for count 1, for constituency Kolar
Assertion failed for 181th run for count 1, for constituency Bagepalli
Assertion failed for 181th run for count 2, for constituency Bagepalli
Assertion failed for 184th run for count 1, for constituency Molakalmuru
Assertion failed for 184th run for count 2, for constituency Molakalmuru
JSON decoding failed for 200th run for count 1, for constituency Nippani
JSON decoding failed for 200th run for count 2, for constituency Nippani
76 th constituency starting
151 th constituency starting
count 1 for run number 215
average time taken for 1 constituency 12.378182662349511


In [125]:
with open('name_merge.json','w',encoding='utf-8') as f:
    json.dump(new_dict,f)